In [21]:
from common_imports import *
from ocr_extractor import extract_text_from_pdf

from getpass import getpass

# LLM Models
from openai import OpenAI
import openai
from pydantic import BaseModel, Field, AfterValidator, WithJsonSchema
import instructor

# Typing
from typing import Optional, Iterable, List, Annotated
from datetime import date

# Print
from pprint import pprint
import json


In [22]:
# Setup OpenAI
if os.getenv("OPEN_AI_KEY") is None:
    if any(['VSCODE' in x for x in os.environ.keys()]):
        print("Please enter password in the VS Code prompt at the top of your VS Code window!")
    
    os.environ['OPENAI_API_KEY'] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
    openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "Doesn't look like an API key"
print("OpenAI API key configured")

    

Please enter password in the VS Code prompt at the top of your VS Code window!
OpenAI API key configured


# Create the structure of a resume

Create the structure to help LLMs scrape necessary data from the resume

In [23]:
def day_validator(day: int):
    if day == None:
        day = 15
    if day > 31 or day < 0:
        raise ValueError("Day not in range")
    return day
    

def month_validator(month: int):
    if month == None:
        month = 6
        
    if month > 12 or month < 0:
        raise ValueError("Month not in range")

    return month



class ForgivingDate(BaseModel):
    day: int = Annotated[
        int, 
        AfterValidator(day_validator),
        WithJsonSchema({
            'type': 'int',
            'description': 'the day (optional)'
        })
    ]
    month: int = Annotated[
        int, 
        AfterValidator(month_validator),
        WithJsonSchema({
            'type': 'int',
            'description': 'the month (optional)'
        })
    ]
    year: int

    # def __init__(self, date: date):
    #     self.day = day_validator(date.day)
    #     self.month = month_validator(date.month)
    #     self.year = date.year



class DateRange(BaseModel):
    start: ForgivingDate
    end: ForgivingDate = Field(description='Date of the end', default=ForgivingDate(day=date.today().day, month=date.today().month, year=date.today().year))


class Skill(BaseModel):
    name: str = Field(description="Extract the technical tools in the following text. Technical tools are generally in 2-3 words")
    years: int = Field(description='Years of experience deducted from the (number of days between the dates)/365 and rounded up')




class Experience(BaseModel):
    dates: DateRange
    title: str = Field(description='The name of the role')
    company: str = Field(description='The employer')
    skills: List[Skill]
    description:str = Field(description='spell check all these sentences. Do not summarize anything')


class Education(BaseModel):
    college: str = Field(description='Institution from which the person received their degree')
    
    dates: DateRange


class Contact(BaseModel):
    phone_number: str 
    email: str 
    location: str = Field(
        default_factory=str, 
        description='Complete street address wherever possible.'
        )
    
    
    



class Candidate(BaseModel):
    name: str
    contact: Contact
    education: Education
    
    experiences: List[Experience]

class OptionalCandidate(BaseModel):
    result: Optional[Candidate] = Field(default=None)
    error: bool = Field(default=False)
    message: Optional[str]




# Extracting a PDF 

Extract PDF of a resume for a quick RAG 

In [24]:
# Get instructed
text_chunk = extract_text_from_pdf(SETTINGS['source_pdf_file'])


# install client
client = instructor.patch(OpenAI(), mode=instructor.Mode.MD_JSON)


# extractions
extraction = client.chat.completions.create(
    model=SETTINGS['model']['name'],
    response_model = OptionalCandidate,
    messages=[
        {
            'role': 'system',
            'content': f'Your role is to extract information from the following resume. The present is {date.today()}'
        },
        {
            'role': 'user',
            'content': text_chunk
        }
    ],
) 

INFO - Number of pages: 1
/Users/paramesh/miniconda3/envs/resume-extractor/lib/python3.12/site-packages/pydantic/json_schema.py:2099: PydanticJsonSchemaWarning: Default value typing.Annotated[int, AfterValidator(func=<function day_validator at 0x10c9cd300>), WithJsonSchema(json_schema={'type': 'int', 'description': 'the day (optional)'}, mode=None)] is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/paramesh/miniconda3/envs/resume-extractor/lib/python3.12/site-packages/pydantic/json_schema.py:2099: PydanticJsonSchemaWarning: Default value typing.Annotated[int, AfterValidator(func=<function month_validator at 0x10c9cf9c0>), WithJsonSchema(json_schema={'type': 'int', 'description': 'the month (optional)'}, mode=None)] is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [25]:
json_result = extraction.model_dump_json()
print(json.dumps(json.loads(json_result), indent=4))

{
    "result": {
        "name": "Hemanth Vikash Kannan Rajan",
        "contact": {
            "phone_number": "+1 540-449-8971",
            "email": "hemanthv@vt.edu",
            "location": "Brooklyn NYC - 11226"
        },
        "education": {
            "college": "Virginia Tech",
            "dates": {
                "start": {
                    "day": 1,
                    "month": 8,
                    "year": 2016
                },
                "end": {
                    "day": 1,
                    "month": 5,
                    "year": 2020
                }
            }
        },
        "experiences": [
            {
                "dates": {
                    "start": {
                        "day": 1,
                        "month": 5,
                        "year": 2021
                    },
                    "end": {
                        "day": 21,
                        "month": 3,
                        "year": 2024
               

In [14]:
writeFile =open('../data/example.json', 'w')
writeFile.write(json_result)
writeFile.close()